In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)
dataset_path = 'gdrive/My Drive/Deep Learning/sentiment analysis/'

Mounted at /content/gdrive


In [ ]:
import torch
import random
import numpy as np
from transformers import BertConfig
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
len(tokenizer.vocab)

30522

In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [ ]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.long)

In [ ]:
train_data, valid_data, test_data = data.TabularDataset.splits(
    path=dataset_path, train="sst_train.csv", 
    validation="sst_dev.csv", test="sst_test.csv",format="csv", skip_header=True, 
    fields=[('text', TEXT), ('truth', LABEL)]
)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of valid examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 8544
Number of valid examples: 1101
Number of testing examples: 2210


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(vars(train_data.examples[6]))

{'text': [2074, 1996, 4428, 2920, 1999, 4526, 1996, 21323, 4138, 2791, 1997, 1996, 13425, 1999, 2023, 9610, 10464, 28817, 3217, 1997, 12013, 1998, 2422, 2003, 26137, 1012], 'truth': '5'}


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['just', 'the', 'labour', 'involved', 'in', 'creating', 'the', 'layered', 'rich', '##ness', 'of', 'the', 'imagery', 'in', 'this', 'chi', '##aro', '##scu', '##ro', 'of', 'madness', 'and', 'light', 'is', 'astonishing', '.']


In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f42487f4d08>, {'4': 0, '2': 1, '3': 2, '5': 3, '1': 4})


In [ ]:
BATCH_SIZE = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# keep in mind the sort_key option 
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), sort_key=lambda x: len(x.text),
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=device)
LABEL.vocab.freqs

Counter({'1': 1092, '2': 2218, '3': 1624, '4': 2322, '5': 1288})

In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        #self.config = config
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        #print(embedding_dim)
        bert.config.output_hidden_states = True
        #print(bert.config)
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            out_bert = self.bert(text) #, config = self.config)
            #embedded = out_bert[0]
            #print(embedded.shape)
        #print(len(out_bert[2]))
        last = out_bert[2]
        l_1, l_2, l_3, l_4 = last[-1], last[-2], last[-3], last[-4]
        #print(l_1.shape)
        embedded = torch.cat((l_1,l_2,l_3,l_4), dim = 1)
        #print(embedded.shape)       
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        #print(hidden.shape)
        output = self.out(hidden)
        #print(output.shape)
        #output = [batch size, out dim]
        
        return output

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 5
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,761,221 trainable parameters


In [ ]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,761,221 trainable parameters


In [ ]:
import torch.optim as optim

#optimizer = optim.Adam(model.parameters())

In [ ]:
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)#,weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
import torch.nn.functional as F
def binary_accuracy(preds1, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #print("Here")
    preds, ind= torch.max(F.softmax(preds1),1)
    correct = (ind == y).float()
    acc = correct.sum()/float(len(correct))
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        #print(predictions.shape, batch.truth)
        loss = criterion(predictions, batch.truth)
        #print(loss)
        acc = binary_accuracy(predictions, batch.truth)
        #print(acc)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.truth)
            
            acc = binary_accuracy(predictions, batch.truth)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'bert-model.pt')
    
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


	Train Loss: 1.347 | Train Acc: 40.05%
	 Val. Loss: 1.219 |  Val. Acc: 46.01%
	Train Loss: 1.189 | Train Acc: 47.76%
	 Val. Loss: 1.188 |  Val. Acc: 46.64%
	Train Loss: 1.141 | Train Acc: 50.05%
	 Val. Loss: 1.193 |  Val. Acc: 47.46%
	Train Loss: 1.102 | Train Acc: 52.28%
	 Val. Loss: 1.173 |  Val. Acc: 48.45%
	Train Loss: 1.057 | Train Acc: 53.52%
	 Val. Loss: 1.186 |  Val. Acc: 48.86%


In [ ]:
test_add_notan = []

In [ ]:
model.load_state_dict(torch.load('bert-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


Test Loss: 1.128 | Test Acc: 50.90%


In [ ]:
test_add_notan

[46.919642857142854, 48.16964285714286, 46.517857142857146, 47.58928571428571]

In [ ]:
sum(test_norm_tanh)/len(test_norm_tanh)

47.27678571428571

In [ ]:
sum(test_1)/len(test_1)

47.69642857142857

In [ ]:
sum(test)/len(test)

47.84821428571429

In [ ]:
test_tanh

[46.964285714285715,
 47.767857142857146,
 48.16964285714286,
 47.5,
 47.00892857142857,
 46.60714285714286]

In [ ]:
sum(test_tanh[:-1])/len(test_tanh[:-1])

47.482142857142854

In [ ]:
sum(test_add)/len(test_add)

47.481142857142856

In [ ]:
sum(test_add_notan)/len(test_add_notan)

47.29910714285714